In [ ]:
!pip install llama-cpp-python
!pip install langchain
!pip install chromadb

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain,VectorDBQA
from langchain.text_splitter import MarkdownTextSplitter
from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
markdown_text = """
# 自定义组件
## 1. 组件模板和样式
类似于页面，自定义组件拥有自己的 FXML 模板和 FTSS 样式。
### 1.1 组件模板
组件模板的写法与页面模板相同。组件模板与组件数据结合后生成的节点树，将被插入到组件的引用位置上。

在组件模板中可以提供一个 `<slot>` 节点，用于承载组件引用时提供的子节点。​

**代码示例**
```html
<!-- 组件模板 -->
<view class="wrapper">
  <view>这里是组件的内部节点</view>
  <slot></slot>
</view>
<!-- 引用组件的页面模板 -->
<view>
  <component-tag-name>
    <!-- 这部分内容将被放置在组件 <slot> 的位置上 -->
    <view>这里是插入到组件slot中的内容</view>
  </component-tag-name>
</view>
```
"""
markdown_splitter = MarkdownTextSplitter(chunk_size=100, chunk_overlap=0)
docs = markdown_splitter.create_documents([markdown_text])

# 初始化 openai 的 embeddings 对象
embeddings = LlamaCppEmbeddings(model_path="./llama.cpp/zh-models/7B/ggml-model-q4_0.bin")
# 将 document 通过 openai 的 embeddings 对象计算 embedding 向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(docs, embeddings)

llm = LlamaCpp(model_path="./llama.cpp/zh-models/7B/ggml-model-q4_0.bin")


In [ ]:
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch,return_source_documents=True)
# 进行问答
result = qa({"query": "组件模板的写法？"})
print(result)